In [3]:
pip install python-whois


[notice] A new release of pip available: 22.3 -> 23.1
[notice] To update, run: C:\Users\tnish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import whois


def abnormal_features(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
        response = requests.get(url, headers=headers, timeout=5)
    except:
        return 0, 0, 0, 0, 0

    if response.status_code != 200:
        return 0, 0, 0, 0, 0

    parsed_url = urlparse(url)
    domain = parsed_url.netloc

    external_links = []
    external_count = 0
    total_count = 0
    meta_count = 0
    script_count = 0
    link_count = 0

    soup = BeautifulSoup(response.text, "html.parser")

    for link in soup.find_all("a"):
        if "href" in link.attrs:
            href = link["href"]
            if "http" in href:
                total_count += 1
                if domain not in href:
                    external_links.append(href)
                    external_count += 1

    meta_count = response.text.count("<meta")
    script_count = response.text.count("<script")
    link_count = response.text.count("<link")

    for link in response.text.split("src="):
        if "http" in link:
            total_count += 1
            if domain not in link:
                external_links.append(link.strip("'\""))
                external_count += 1

    try:
        request_url_percentage = (len(external_links) / total_count) * 100
    except ZeroDivisionError:
        request_url_percentage = 0

    if request_url_percentage < 22:
        request_url_feature = 2
    elif request_url_percentage >= 22 and request_url_percentage <= 61:
        request_url_feature = 1
    else:
        request_url_feature = 0

    try:
        anchor_url_percentage = (external_count / total_count) * 100
    except ZeroDivisionError:
        anchor_url_percentage = 0

    if anchor_url_percentage < 31:
        anchor_url_feature = 2
    elif anchor_url_percentage >= 31 and anchor_url_percentage <= 67:
        anchor_url_feature = 1
    else:
        anchor_url_feature = 0

    try:
        link_percentage = (external_count / total_count) * 100
    except ZeroDivisionError:
        link_percentage = 0

    if link_percentage < 17:
        meta_script_link_feature = 2
    elif link_percentage >= 17 and link_percentage <= 81:
        meta_script_link_feature = 1
    else:
        meta_script_link_feature = 0

    form_action = ""
    for line in response.text.split("\n"):
        if "<form" in line:
            if 'action="' in line:
                form_action = line.split('action="')[1].split('"')[0]
            else:
                form_action = url
            break

    if not form_action or form_action == "about:blank":
        sfh_feature = 0
    else:
        form_action_domain = urlparse(form_action).netloc
        if form_action_domain and form_action_domain != domain:
            sfh_feature = 1
        else:
            sfh_feature = 2

    try:
        domain_info = whois.whois(domain)
        if domain_info is None:
            domain_check = 0
        else:
            domain_check = 1
    except:
        domain_check = 0

    return request_url_feature, anchor_url_feature, meta_script_link_feature, sfh_feature, domain_check



# import pandas as pd
# from multiprocessing import Pool

# def process_url(url):
#     # function to process a single URL and return a tuple of features
#     return abnormal_features(url)

# # create example dataframe
# df = pd.read_csv('Class1.csv')
# df = df.head(100)

# # create a Pool with 4 processes
# with Pool(processes=4) as pool:
#     # apply the process_url function to the final_url column using the pool
#     results = pool.map(process_url, df['final_url'])

# # convert the list of tuples to a dataframe
# df_features = pd.DataFrame(results, columns=['request_url_feature', 'anchor_url_feature', 'meta_script_link_feature', 'sfh_feature', 'domain_check'])
# print(df_features)
# # concatenate original dataframe and new dataframe along the columns axis
# df_concat = pd.concat([df, df_features], axis=1)

# # write concatenated dataframe to CSV file
# df_concat.to_csv('Class1_with_features.csv', index=False)


In [8]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def process_url(url):
# function to process a single URL and return a tuple of features
    return abnormal_features(url)

        


# create example dataframe
df = pd.read_csv('Class1.csv')
# df = df.head(100)
test = len(df)
print(test)
df = df.loc[600000 :test]

# create a ThreadPoolExecutor with 2 threads
with ThreadPoolExecutor(max_workers=1000) as executor:
    # apply the process_url function to the final_url column using the executor
    futures = [executor.submit(process_url, url) for url in df['final_url']]
    
    # retrieve the results from the futures as they complete
    # results = [future.result() for future in futures]
    # retrieve the results from the futures as they complete
    results = []
    counter = 0 # initialize counter variable
    for future in futures:
        results.append(future.result())

# convert the list of tuples to a dataframe
df_features = pd.DataFrame(results, columns=['request_url_feature', 'anchor_url_feature', 'meta_script_link_feature', 'sfh_feature', 'domain_check'])

# concatenate original dataframe and new dataframe along the columns axis
df_concat = pd.concat([df.reset_index(drop=True), df_features.reset_index(drop=True)], axis=1)

# write concatenated dataframe to CSV file
df_concat.to_csv('Class1_with_features 600000: EOF.csv', index=False)


C:\Users\tnish\AppData\Local\Temp\ipykernel_6912\3114297585.py:12: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Class1.csv')


640178
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failedError trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed

Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno

C:\Users\tnish\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failedError trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed

Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
Error trying to connect to socket: closing socket - [Errno 11001]

HTML JS BASED FEATURES

In [4]:
pip install requests


     ---------------------------------------- 62.8/62.8 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 96.7/96.7 kB ? eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB 3.4 MB/s eta 0:00:00
     -------------------------------------- 140.9/140.9 kB 8.2 MB/s eta 0:00:00
     -------------------------------------- 155.3/155.3 kB 9.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: C:\Users\tnish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
pip install bs4

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 143.0/143.0 kB 4.3 MB/s eta 0:00:00
  Running setup.py install for bs4: started
  Running setup.py install for bs4: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: bs4 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: C:\Users\tnish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import requests
from bs4 import BeautifulSoup

class HTML_JS_Features:
    def redirect_feature(self,url):
        try:
            response = requests.get(url, allow_redirects=True)
        except:
            return 0

        num_redirects = len(response.history)

        if num_redirects <= 1:
            return 2
        elif num_redirects >= 2 and num_redirects < 4:
            return 1
        else:
            return 0

    def check_status_bar(self, url):
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            status_bar = soup.find('a').get('onmouseover')
            if status_bar and 'window.status' in status_bar.lower():
                return 0
            else:
                return 1
        except:
            return 0


    def check_right_click(self, url):
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            script_tags = soup.find_all('script')
            for tag in script_tags:
                if 'event.button==2' in tag.text:
                    return 0
            return 1
        except:
            return 0

    def popup_feature(self, url):
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Check if there is a popup window
            popup = soup.find('div', {'class': 'popup-window'})
            if popup is not None:
                # Check if the popup contains text fields
                text_fields = popup.find_all('input', {'type': 'text'})
                if len(text_fields) > 0:
                    return 0
        except:
            # print("Error: ", e)
            return 0

        return 1



    def check_iframe(self,url):
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            iframes = soup.find_all('iframe')
            for iframe in iframes:
                if 'frameborder' in iframe.attrs and iframe.attrs['frameborder'] == '0':
                    return 0
            return 1
        except:
            return 0

In [15]:
# import pandas as pd
checker2 = HTML_JS_Features()
# df = pd.read_csv('Class0.csv')
# df = df.head(11) 

# apply the AbnormalFeatures methods to each row in the dataframe
# df['redirect_feature'] = df['final_url'].apply(checker2.redirect_feature) --
# df['check_status_bar'] = df['final_url'].apply(checker2.check_status_bar)
# df['check_right_click'] = df['final_url'].apply(checker2.check_right_click)
# df['popup_feature'] = df['final_url'].apply(checker2.popup_feature)
# df['check_iframe'] = df['final_url'].apply(checker2.check_iframe)


# checker2.redirect_feature('https://colab.research.google.com/drive/17X3XT3utlHzyyBLA3QtyLQ-_DESOSlZB#scrollTo=vdByi-EngXmf')
# checker2.check_status_bar('https://colab.research.google.com/drive/17X3XT3utlHzyyBLA3QtyLQ-_DESOSlZB#scrollTo=vdByi-EngXmf')
# checker2.check_right_click('https://colab.research.google.com/drive/17X3XT3utlHzyyBLA3QtyLQ-_DESOSlZB#scrollTo=vdByi-EngXmf')
# checker2.popup_feature('https://colab.research.google.com/drive/17X3XT3utlHzyyBLA3QtyLQ-_DESOSlZB#scrollTo=vdByi-EngXmf')
checker2.check_iframe('https://colab.research.google.com/drive/17X3XT3utlHzyyBLA3QtyLQ-_DESOSlZB#scrollTo=vdByi-EngXmf')

1

In [ ]:
import concurrent.futures
import pandas as pd
import requests
from bs4 import BeautifulSoup

def abnormal_features(url):
    redirect_feature_result = None
    check_status_bar_result = None
    check_right_click_result = None
    popup_feature_result = None
    check_iframe_result = None

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        status_bar = soup.find('a').get('onmouseover')

        num_redirects = len(response.history)
        if num_redirects <= 1:
            redirect_feature_result = 2
        elif num_redirects >= 2 and num_redirects < 4:
            redirect_feature_result = 1
        else:
            redirect_feature_result = 0

        if status_bar and 'window.status' in status_bar.lower():
            check_status_bar_result = 0
        else:
            check_status_bar_result = 1

        script_tags = soup.find_all('script')
        for tag in script_tags:
            if 'event.button==2' in tag.text:
                check_right_click_result = 0
                break
        else:
            check_right_click_result = 1

        popup = soup.find('div', {'class': 'popup-window'})
        if popup is not None:
            text_fields = popup.find_all('input', {'type': 'text'})
            if len(text_fields) > 0:
                popup_feature_result = 0
            else:
                popup_feature_result = 1
        else:
            popup_feature_result = 1

        iframes = soup.find_all('iframe')
        for iframe in iframes:
            if 'frameborder' in iframe.attrs and iframe.attrs['frameborder'] == '0':
                check_iframe_result = 0
                break
        else:
            check_iframe_result = 1

    except:
        redirect_feature_result = 0
        check_status_bar_result = 0
        check_right_click_result = 0
        popup_feature_result = 0
        check_iframe_result = 0

    return (redirect_feature_result, check_status_bar_result, check_right_click_result, popup_feature_result, check_iframe_result)


# create example dataframe
df = pd.read_csv('Class1.csv')
df = df.loc[600000:]

# create a ThreadPoolExecutor with 1000 threads
with concurrent.futures.ThreadPoolExecutor(max_workers=1000) as executor:
    # apply the abnormal_features function to the final_url column using the executor
    futures = [executor.submit(abnormal_features, url) for url in df['final_url']]
    
    # retrieve the results from the futures as they complete
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

# convert the list of tuples to a dataframe
df_features = pd.DataFrame(results, columns=['redirect_feature', 'check_status_bar', 'check_right_click', 'popup_feature', 'check_iframe'])

# concatenate original dataframe and new dataframe along the columns axis
df_concat = pd.concat([df.reset_index(drop=True), df_features.reset_index(drop=True)], axis=1)

# write concatenated dataframe to CSV file
df_concat.to_csv('Class1_with_features 600000: EOF.csv', index=False)


In [1]:
import pandas as pd

df = pd.read_csv('Class1.csv')
df['domain_registration_length'] = df['domain_registration_length'].fillna(0)

df.to_csv('Final_URLs.csv')

C:\Users\tnish\AppData\Local\Temp\ipykernel_24808\4181412394.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Class1.csv')
